### Libs

In [1]:
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Data

In [2]:
with open('dataset\cvrp_castanhal_completo.json') as json_file:
    raw_data = json.load(json_file)

In [3]:
raw_data.keys()

dict_keys(['name', 'region', 'origin', 'vehicle_capacity', 'deliveries'])

In [4]:
raw_data["origin"]

{'lng': -47.93317089927402, 'lat': -1.2923387357484621}

In [5]:
raw_data["vehicle_capacity"]

180

In [6]:
raw_data["deliveries"][0]

{'id': '245aed8b07925496f14cc83f85a95318',
 'point': {'lng': -48.083796008609376, 'lat': -1.3859328704379723},
 'size': 8}

### General Class

In [7]:
class Coordinate:
    def __init__(self, lat, lng):
        self.lat = lat
        self.lng = lng

In [8]:
class Delivery:
    def __init__(self, raw_delivery, index):
        self.id = index+1
        self.point = Coordinate(str(raw_delivery["point"]["lat"]), str(raw_delivery["point"]["lng"]))
        self.size = raw_delivery["size"]

In [9]:
class DepotNet:
    def __init__(self, raw_data):
        self.name = raw_data["name"]
        self.region = raw_data["region"]
        self.origin = Coordinate(str(raw_data["origin"]["lat"]), str(raw_data["origin"]["lng"]))
        self.vehicle_capacity = raw_data["vehicle_capacity"]
        self.deliveries = [Delivery(raw_data["deliveries"][i], i) for i in range(len(raw_data["deliveries"]))]

### Tabu Search

In [10]:
#Class Tabu Search for the problem of the CVRP
class TabuSearch:
    def __init__(self, depot_net, max_iterations, n_neighbors, tabu_list_size):
        self.depot_net = depot_net
        self.max_iterations = max_iterations
        self.n_neighbors = n_neighbors
        self.tabu_list_size = tabu_list_size
        self.solution = []
        self.best_solution = []
        self.best_solutions = pd.DataFrame(columns=[f"x_{i}" for i in range(len(self.depot_net.deliveries))]+["y"])
        self.best_neighborhood = pd.DataFrame(columns=[f"x_{i}" for i in range(len(self.depot_net.deliveries))]+["y"])
        self.tabu_list = []

    def generate_solution_initial(self):
        n = len(self.depot_net.deliveries)
        solution = np.random.choice([i for i in range(n)], n, replace=False)
        return [self.depot_net.deliveries[idx].id for idx in solution]
    
    def udpate_best_solutions(self, solution, solution_cost):
        s = pd.Series([self.depot_net.deliveries[idx-1].id for idx in solution]+[solution_cost],index=[f"x_{i}" for i in range(len(solution))]+["y"])
        self.best_solutions = self.best_solutions.append(s, ignore_index=True)
        df = self.best_solutions.iloc[:, :-1].astype(int)
        df["y"] = self.best_solutions["y"]
        self.best_solutions = df

    def udpate_best_neighborhood(self, solution, solution_cost):
        s = pd.Series([self.depot_net.deliveries[idx-1].id for idx in solution]+[solution_cost],index=[f"x_{i}" for i in range(len(solution))]+["y"])
        self.best_neighborhood = self.best_neighborhood.append(s, ignore_index=True)
        df = self.best_neighborhood.iloc[:, :-1].astype(int)
        df["y"] = self.best_neighborhood["y"]
        self.best_neighborhood = df
    
    def solution_cost(self, solution, save=False):
        solution_cost = 0
        route_list = [f"{self.depot_net.origin.lng},{self.depot_net.origin.lat}"]
        weight = 0
        for delivery in [self.depot_net.deliveries[idx-1] for idx in solution]:
            weight += delivery.size
            if weight <= self.depot_net.vehicle_capacity:
                route_list.append(f"{delivery.point.lng},{delivery.point.lat}")
            else:
                route_list.append(f"{self.depot_net.origin.lng},{self.depot_net.origin.lat}")
                route_list.append(f"{delivery.point.lng},{delivery.point.lat}")
                weight = 0
        route_list.append(f"{self.depot_net.origin.lng},{self.depot_net.origin.lat}")
        route_str = ";".join(route_list)
        query = f"https://router.project-osrm.org/route/v1/driving/{route_str}?overview=false"
        #print(query)
        response = requests.get(query)
        data = response.json()
        #salvar resposta da requisição em arquivo no formato json
        if save:
            with open('best_routes_cvrp.json', 'w') as outfile:
                json.dump(data, outfile)
        solution_cost = data["routes"][0]["distance"]
        return solution_cost
    
    def swap(self, solution):
        i, j = np.random.choice(len(solution)-1, 2, replace=False)
        solution[i], solution[j] = solution[j], solution[i]
        return solution
    
    def diference_solutions(self, solution, best_neighbor):
        indexs = [i for i in range(len(solution)) if solution[i] != best_neighbor[i]]
        return indexs
    
    def generate_neighbors(self, iteration):
        neighbors = []
        neighbors_cost = []
        for _ in range(self.n_neighbors):
            neighbor = self.swap(self.best_solution.copy())
            while neighbor in neighbors:
                neighbor = self.swap(self.solution.copy())
            neighbor_cost = self.solution_cost(neighbor)
            if neighbors_cost == []:
                neighbors.append(neighbor)
                neighbors_cost.append(neighbor_cost)
            else:
                for k in range(len(neighbors_cost)):
                    if neighbor_cost < neighbors_cost[k]:
                        neighbors.insert(k, neighbor)
                        neighbors_cost.insert(k, neighbor_cost)
                        break
                    if k == len(neighbors_cost)-1:
                        neighbors.append(neighbor)
                        neighbors_cost.append(neighbor_cost)
        self.update_tabu_list(neighbors, neighbors_cost, iteration)
    
    def update_tabu_list(self, neighbors, neighbors_cost, iteration):
        if neighbors_cost[0] < self.solution_cost(self.best_solution):
            if len(self.tabu_list) == self.tabu_list_size:
                self.tabu_list.pop(0)
            self.tabu_list.append(self.diference_solutions(self.solution, neighbors[0]))
            self.solution = neighbors[0]
            #print(f'Mudou para melhor na iteração {iteration}')
            self.best_solution = neighbors[0]
        else:
            for l in range(self.n_neighbors):
                indexs_swap = self.diference_solutions(self.solution, neighbors[l])
                if indexs_swap in self.tabu_list:
                    continue
                else:
                    if len(self.tabu_list) == self.tabu_list_size:
                        self.tabu_list.pop(0)
                    self.tabu_list.append(indexs_swap)
                    self.solution = neighbors[l]
                    #print(f'Mudou para pior na iteração {iteration}')
                    break
        self.udpate_best_neighborhood(neighbors[0], neighbors_cost[0])

    def run(self):
        self.solution = self.generate_solution_initial()
        self.best_solution = self.solution
        print(f'Solução inicial:\n{self.solution}{self.solution_cost(self.best_solution)}')
        for i in range(self.max_iterations):
            print(f'Iteração {i+1}') 
            self.generate_neighbors(i)
            self.udpate_best_solutions(self.best_solution, self.solution_cost(self.best_solution))
        print(f'Solução Final:\n{self.best_solution}{self.solution_cost(self.best_solution)}')

### Run

In [11]:
depot_net = DepotNet(raw_data)
max_iterations = 1000
n_neighbors = 100
tabu_list_size = 50

tabu = TabuSearch(depot_net, max_iterations, n_neighbors, tabu_list_size)
tabu.run()
tabu.solution_cost(tabu.best_solution, save=True)

Solução inicial:
[25, 18, 11, 33, 16, 12, 34, 28, 22, 27, 8, 35, 5, 19, 30, 15, 13, 3, 20, 1, 23, 21, 4, 31, 14, 32, 10, 9, 6, 17, 29, 24, 2, 26, 7]527168.2
Iteração 1
Iteração 2
Iteração 3
Iteração 4
Iteração 5
Iteração 6
Iteração 7
Iteração 8
Iteração 9
Iteração 10
Iteração 11
Iteração 12
Iteração 13
Iteração 14
Iteração 15
Iteração 16
Iteração 17
Iteração 18
Iteração 19
Iteração 20
Iteração 21
Iteração 22
Iteração 23
Iteração 24
Iteração 25
Iteração 26
Iteração 27
Iteração 28
Iteração 29
Iteração 30
Iteração 31
Iteração 32
Iteração 33
Iteração 34
Iteração 35
Iteração 36
Iteração 37
Iteração 38
Iteração 39
Iteração 40
Iteração 41
Iteração 42
Iteração 43
Iteração 44
Iteração 45
Iteração 46
Iteração 47
Iteração 48
Iteração 49
Iteração 50
Iteração 51
Iteração 52
Iteração 53
Iteração 54
Iteração 55
Iteração 56
Iteração 57
Iteração 58
Iteração 59
Iteração 60
Iteração 61
Iteração 62
Iteração 63
Iteração 64
Iteração 65
Iteração 66
Iteração 67
Iteração 68
Iteração 69
Iteração 70
Iteração 71


147347.6

### Analysis

In [12]:
tabu.solution_cost(tabu.best_solution, save=True)

with open('best_routes_cvrp.json', 'r') as outfile:
    data = json.load(outfile)     

In [13]:
print(f'Melhores soluções das vizinhanças:')
display(tabu.best_neighborhood)
index_best_neighborhood = tabu.best_neighborhood.loc[tabu.best_neighborhood["y"] == tabu.best_neighborhood["y"].min()].index
print(f'Melhor(res) solução(ões) apresentada(s) no(s) índece(s):\n{index_best_neighborhood.astype(int).tolist()}')

Melhores soluções das vizinhanças:


,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_26,x_27,x_28,x_29,x_30,x_31,x_32,x_33,x_34,y
0,25,18,11,33,16,12,34,28,22,27,...,10,9,6,17,29,24,2,26,7,465667.5
1,25,18,11,33,16,12,34,28,22,27,...,10,9,6,17,29,24,2,26,7,461471.2
2,25,18,11,33,16,12,34,28,22,27,...,10,9,31,17,29,24,2,26,7,400163.9
3,25,18,11,33,24,12,34,28,22,27,...,10,9,31,17,29,16,2,26,7,396937.3
4,25,18,11,17,24,12,34,28,22,27,...,10,9,31,33,29,16,2,26,7,387916.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9,18,14,17,24,12,26,28,10,21,...,34,16,20,33,31,32,35,30,7,148303.1
96,9,18,14,17,24,12,26,28,10,21,...,29,16,20,33,31,32,35,30,7,147390.4
97,9,18,14,17,27,12,26,28,10,21,...,29,16,20,33,31,32,35,30,7,149879.3
98,9,18,14,22,24,12,26,28,10,21,...,29,16,20,33,31,32,35,30,7,149038.6


Melhor(res) solução(ões) apresentada(s) no(s) índece(s):
[93]


In [14]:
print(f'Melhores soluções globais por iteração:')
display(tabu.best_solutions)
index_best_solutions = tabu.best_solutions.loc[tabu.best_solutions["y"] == tabu.best_solutions["y"].min()].index
print(f'Melhor(res) solução(ões) apresentada(s) no(s) índece(s):\n{index_best_solutions.astype(int).tolist()}')

Melhores soluções globais por iteração:


,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_26,x_27,x_28,x_29,x_30,x_31,x_32,x_33,x_34,y
0,25,18,11,33,16,12,34,28,22,27,...,10,9,6,17,29,24,2,26,7,465667.5
1,25,18,11,33,16,12,34,28,22,27,...,10,9,6,17,29,24,2,26,7,461471.2
2,25,18,11,33,16,12,34,28,22,27,...,10,9,31,17,29,24,2,26,7,400163.9
3,25,18,11,33,24,12,34,28,22,27,...,10,9,31,17,29,16,2,26,7,396937.3
4,25,18,11,17,24,12,34,28,22,27,...,10,9,31,33,29,16,2,26,7,387916.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9,18,14,17,24,12,26,28,10,21,...,29,16,20,33,31,32,35,30,7,147347.6
96,9,18,14,17,24,12,26,28,10,21,...,29,16,20,33,31,32,35,30,7,147347.6
97,9,18,14,17,24,12,26,28,10,21,...,29,16,20,33,31,32,35,30,7,147347.6
98,9,18,14,17,24,12,26,28,10,21,...,29,16,20,33,31,32,35,30,7,147347.6


Melhor(res) solução(ões) apresentada(s) no(s) índece(s):
[93, 94, 95, 96, 97, 98, 99]


In [15]:
#plot curva de minimização
plt.plot(tabu.best_solutions["x"], tabu.best_solutions["y"])
plt.xlabel('Iteração')
plt.ylabel('Custo')
plt.title('Curva de minimização')
plt.show()

KeyError: 'x'